In [1]:
import pandas as pd
import numpy as np

# Case A: 기존 예측 기반 의사결정 로그 (Core 5)
caseA_df = pd.read_csv("../data_csv/core5_decision_log.csv")

# Case B: μHSM 상태 모니터 (Core 6)
mu_df = pd.read_csv("../data_csv/muHSM_state_monitor.csv")

print("Case A columns:", caseA_df.columns.tolist())
print("Case B columns:", mu_df.columns.tolist())

Case A columns: ['asset_id', 'date', 't_index', 'state_value', 'degradation_rate', 'risk_group', 'intervention_flag', 'stabilized']
Case B columns: ['user_id', 'date', 'HSI', 'HDR', 'recovery_margin', 'observability_score']


Case A 정리 (Prediction-based)
입력:
degradation_rate + risk_group
(상태 맥락 없음)

In [2]:
caseA = caseA_df.copy()
caseA = caseA.sort_values(["asset_id", "t_index"]).reset_index(drop=True)
caseA.head()

,asset_id,date,t_index,state_value,degradation_rate,risk_group,intervention_flag,stabilized
0,1503960366,2016-04-12,0,-0.002802,NaN,mid,0,False
1,1503960366,2016-04-13,1,0.452790,NaN,mid,0,False
2,1503960366,2016-04-14,2,NaN,NaN,mid,0,False
3,1503960366,2016-04-15,3,NaN,NaN,mid,0,False
4,1503960366,2016-04-16,4,NaN,NaN,mid,0,False


Case B 정리 (μHSM-based)
입력:
HSI + HDR + RM + OBS
(상태 맥락 포함)

In [3]:
caseB = mu_df.copy()

# 컬럼 통일
caseB = caseB.rename(columns={
    "user_id": "asset_id",
    "HDR": "degradation_rate"
})

caseB = caseB.sort_values(["asset_id", "date"]).reset_index(drop=True)

caseB.head()

,asset_id,date,HSI,degradation_rate,recovery_margin,observability_score
0,1503960366,2016-04-12,-0.002802,0.000000,NaN,0.63871
1,1503960366,2016-04-13,0.452790,0.056949,NaN,0.63871
2,1503960366,2016-04-14,NaN,NaN,NaN,0.63871
3,1503960366,2016-04-15,NaN,NaN,NaN,0.63871
4,1503960366,2016-04-16,NaN,NaN,0.2,0.63871


Case B 의사결정 규칙 정의 (규칙 수 동일)
	•	Case A와 규칙 개수 동일
	•	threshold 늘리지 않음

In [4]:
def muHSM_intervention_rule(row):
    if (
        row["degradation_rate"] < -0.05 and
        row["recovery_margin"] < 0.3 and
        row["observability_score"] > 0.6
    ):
        return 1
    return 0

In [5]:
caseB["intervention_flag"] = caseB.apply(
    muHSM_intervention_rule,
    axis=1
)

caseB[["asset_id", "degradation_rate", "recovery_margin",
       "observability_score", "intervention_flag"]].head()

,asset_id,degradation_rate,recovery_margin,observability_score,intervention_flag
0,1503960366,0.000000,NaN,0.63871,0
1,1503960366,0.056949,NaN,0.63871,0
2,1503960366,NaN,NaN,0.63871,0
3,1503960366,NaN,NaN,0.63871,0
4,1503960366,NaN,0.2,0.63871,0


안정화(stabilization) 계산 함수 (공통)

In [6]:
def compute_stabilization(df, state_col, group_col="asset_id", window=7):
    df = df.copy()

    df["post_state"] = (
        df
        .groupby(group_col)[state_col]
        .shift(-window)
    )

    df["stabilized"] = (
        df["post_state"] - df[state_col]
    ) > 0

    return df

Case A 안정화 결과

In [7]:
caseA_eval = compute_stabilization(
    caseA,
    state_col="state_value",
    group_col="asset_id",
    window=7
)

caseA_eval.groupby("intervention_flag")["stabilized"].mean()

intervention_flag
0    0.119298
1    0.590909
Name: stabilized, dtype: float64

Case B 안정화 결과

In [8]:
caseB_eval = compute_stabilization(
    caseB,
    state_col="HSI",
    group_col="asset_id",
    window=7
)

caseB_eval.groupby("intervention_flag")["stabilized"].mean()

intervention_flag
0    0.149400
1    0.653846
Name: stabilized, dtype: float64

불필요한 개입(False Intervention) 비교

In [9]:
false_A = caseA_eval[
    (caseA_eval["intervention_flag"] == 1) &
    (caseA_eval["stabilized"] == False)
]

false_B = caseB_eval[
    (caseB_eval["intervention_flag"] == 1) &
    (caseB_eval["stabilized"] == False)
]

print("Case A false intervention rate:",
      len(false_A) / max(len(caseA_eval[caseA_eval["intervention_flag"] == 1]), 1))

print("Case B false intervention rate:",
      len(false_B) / max(len(caseB_eval[caseB_eval["intervention_flag"] == 1]), 1))

Case A false intervention rate: 0.4090909090909091
Case B false intervention rate: 0.34615384615384615


개입 빈도 안정성 (토글 빈도)

In [10]:
def intervention_toggle_rate(df, group_col="asset_id"):
    toggle_rates = []

    for _, g in df.groupby(group_col):
        flags = g["intervention_flag"].values
        toggles = np.sum(flags[1:] != flags[:-1])
        toggle_rates.append(toggles / max(len(flags), 1))

    return np.mean(toggle_rates)

In [11]:
print("Case A toggle rate:", intervention_toggle_rate(caseA_eval))
print("Case B toggle rate:", intervention_toggle_rate(caseB_eval))

Case A toggle rate: 0.09324931446660269
Case B toggle rate: 0.029665316762090957


Core 7 요약 테이블

In [12]:
summary = pd.DataFrame({
    "Case": ["Prediction-based (A)", "μHSM-based (B)"],
    "Stabilization_when_intervened": [
        caseA_eval.groupby("intervention_flag")["stabilized"].mean().get(1, np.nan),
        caseB_eval.groupby("intervention_flag")["stabilized"].mean().get(1, np.nan)
    ],
    "False_intervention_rate": [
        len(false_A) / max(len(caseA_eval[caseA_eval["intervention_flag"] == 1]), 1),
        len(false_B) / max(len(caseB_eval[caseB_eval["intervention_flag"] == 1]), 1)
    ],
    "Intervention_toggle_rate": [
        intervention_toggle_rate(caseA_eval),
        intervention_toggle_rate(caseB_eval)
    ]
})

summary

,Case,Stabilization_when_intervened,False_intervention_rate,Intervention_toggle_rate
0,Prediction-based (A),0.590909,0.409091,0.093249
1,μHSM-based (B),0.653846,0.346154,0.029665
